In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the dataset
data = pd.read_csv("final_dataset.csv")

# Filter data for training with fitzpatrick_scale values 3 and 4
train_data = data[data['fitzpatrick_scale'].isin([3, 4])]

# Filter data for testing with fitzpatrick_scale values 1, 2, 5, and 6
test_data = data[data['fitzpatrick_scale'].isin([1, 2, 5, 6])]

# Define the image data generator with augmentation (optional)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define the input image size and batch size
img_size = (224, 224)
batch_size = 32

# Create the generators for training and testing data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
)

# Define the InceptionNet model and load pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer with 1024 neurons
x = Dense(1024, activation='relu')(x)

# Add a final output layer with one neuron per class
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with categorical cross-entropy loss and Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data
history = model.fit(train_generator, epochs=20, validation_data=test_generator)

# Evaluate the model on the testing data
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)


Found 3984 validated image filenames belonging to 114 classes.
Found 8231 validated image filenames belonging to 114 classes.
Epoch 1/20
125/125 [==============================] - 1048s 8s/step - loss: 4.4994 - accuracy: 0.0610 - val_loss: 4.3169 - val_accuracy: 0.0798
Epoch 2/20
125/125 [==============================] - 1007s 8s/step - loss: 3.8614 - accuracy: 0.1273 - val_loss: 4.1122 - val_accuracy: 0.1030
Epoch 3/20
125/125 [==============================] - 996s 8s/step - loss: 3.5248 - accuracy: 0.1870 - val_loss: 4.0514 - val_accuracy: 0.1170
Epoch 4/20
125/125 [==============================] - 1005s 8s/step - loss: 3.2551 - accuracy: 0.2221 - val_loss: 4.0745 - val_accuracy: 0.1268
Epoch 5/20
125/125 [==============================] - 1009s 8s/step - loss: 3.0151 - accuracy: 0.2754 - val_loss: 4.1192 - val_accuracy: 0.1209
Epoch 6/20
125/125 [==============================] - 1018s 8s/step - loss: 2.8234 - accuracy: 0.3057 - val_loss: 4.1017 - val_accuracy: 0.1236
Epoch 7/20


NameError: name 'accuracy' is not defined

In [2]:
print("Testing accuracy: {:.2f}%".format(test_acc * 100))

Testing accuracy: 13.78%


In [6]:
import numpy as np
# Load test set
test_df = pd.read_csv('final_dataset.csv')

# Create data generator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col="local_filename",
    y_col="label",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Predict labels for test set
y_pred = model.predict(test_generator)
predicted_labels = np.argmax(y_pred, axis=1)
actual_labels = test_generator.classes

# Map label indices to label names
label_map = {v: k for k, v in test_generator.class_indices.items()}
predicted_labels = [label_map[label] for label in predicted_labels]
actual_labels = [label_map[label] for label in actual_labels]

# Get fitzpatrick_scale values for test set
fitzpatrick_scale = test_df['fitzpatrick_scale']

# Combine the predicted labels, actual labels, and fitzpatrick_scale values into a DataFrame
results_df = pd.DataFrame({'predicted_label': predicted_labels, 'actual_label': actual_labels, 'fitzpatrick_scale': fitzpatrick_scale})

# Save the DataFrame to a CSV file
results_df.to_csv('results_InceptionNet_tr3n4.csv', index=False)

# Print individual accuracies for each fitzpatrick scale value
for scale in [1,2,5,6]:
    scale_df = results_df[results_df['fitzpatrick_scale'] == scale]
    total = len(scale_df)
    correct = sum(scale_df['predicted_label'] == scale_df['actual_label'])
    accuracy = correct / total
    print(f"Fitzpatrick Scale {scale}: {accuracy:.2%} ({correct}/{total})")


Found 12215 validated image filenames belonging to 114 classes.
382/382 [==============================] - 1774s 5s/step
Fitzpatrick Scale 1: 1.22% (34/2777)
Fitzpatrick Scale 2: 1.35% (56/4136)
Fitzpatrick Scale 5: 0.90% (8/891)
Fitzpatrick Scale 6: 0.47% (2/427)


In [7]:
#UPDATED CODE
import pandas as pd

# read csv file into a pandas dataframe
df = pd.read_csv("results_InceptionNet_tr3n4.csv")

# compute total count of fitzpatrick_scale
total_count = df.shape[0]

print(total_count)

# compute count of correctly predicted values
correctly_predicted = df[df["actual_label"] == df["predicted_label"]].shape[0]
print(correctly_predicted)

answer = (correctly_predicted/total_count)*100

print(answer)

12215
138
1.1297584936553418


In [8]:
accuracy = 13.78
d_accuracy = 1.13
new = accuracy/d_accuracy

f=[1.22,1.35,0.90,0.47]
l = [None] * 4

for i in range(0,4):
    l[i]=f[i]*new
print(l)

[14.877522123893804, 16.462831858407082, 10.975221238938053, 5.731504424778761]
